# Принципы работы Decision Trees

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
import seaborn as sns
sns.set_theme(context='talk', style='whitegrid')

Создадим игрушечные данные

In [ ]:
x = np.arange(0, 10, 0.1).reshape(-1, 1)
y = np.sin(x)

In [ ]:
train_x, test_x = train_test_split(x, test_size=0.3, random_state=42)
train_y, test_y = train_test_split(y, test_size=0.3, random_state=42)

In [ ]:
plt.scatter(train_x, train_y, color='b', s=10, alpha=0.5, label='fact')
plt.legend()
plt.show()

Попробуем обучить модель линейной легрессии

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()
lr.fit(train_x, train_y)

In [ ]:
train_predicts = lr.predict(train_x)
test_predicts = lr.predict(test_x)

In [ ]:
plt.plot(train_x, train_predicts, color='r', label='lr predict')

plt.scatter(train_x, train_y, color='b', s=10, alpha=0.5, label='fact')
plt.legend()

plt.show()

Ожидаемо получили недообученную модель

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
def run_dt(params):
    dt = DecisionTreeRegressor(**params)
    dt.fit(train_x, train_y)

    train_predicts = dt.predict(train_x)
    test_predicts = dt.predict(test_x)

    sorted_train_x, train_predicts = zip(*sorted(zip(train_x, train_predicts)))
    sorted_test_x, test_predicts = zip(*sorted(zip(test_x, test_predicts)))

    # plt.figure(figsize=(16, 9))

    plt.plot(sorted_train_x, train_predicts, color='r', label='dt predict')
    # plt.plot(sorted_test_x, test_predicts, color='r')

    plt.scatter(train_x, train_y, color='b', s=10, alpha=0.5, label='fact')
    # plt.scatter(test_x, test_y, color='r', s=10, alpha=0.5)
    plt.legend()
    plt.show()
    return dt

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
dt = run_dt({'max_depth': 6})

In [ ]:
plt.figure(figsize=(16, 9))
plot_tree(dt)
plt.savefig('tree.png', dpi=600)
plt.show()

In [ ]:
train_y[train_x <= 0.3].shape, train_y[train_x <= 0.3].mean()

In [ ]:
oot_x = np.arange(10, 15, 0.1).reshape(-1, 1)

In [ ]:
oot_predicts = dt.predict(oot_x)

In [ ]:
plt.scatter(oot_x, np.sin(oot_x), color='b', s=10)


plt.plot(oot_x, oot_predicts, color='r', label='dt')
plt.plot(oot_x, lr.predict(oot_x), color='g', label='lr')
plt.legend()
plt.show()

## Применим к задаче

In [ ]:
# Качаем данные
# !gdown 1nCHCT5XWio5fSN0mYNwRbEzTGL_sIcN4

In [ ]:
#Считываем скачанный csv файл
df = pd.read_csv('loan_data.csv')

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

Заменим категориальные переменные

In [ ]:
for col in df.select_dtypes(include='object').columns:
  df[col] = df[col].astype('category').cat.codes

In [ ]:
df.dtypes

In [ ]:
target_col = 'loan_status'

In [ ]:
train, test = train_test_split(df, test_size = 0.3, random_state=42)

In [ ]:
display(train)

In [ ]:
def entropy(p):
    try:
        return -p * np.log2(p) - (1-p) * np.log2(1-p)
    except:
        return 1

In [ ]:
def gini_impurity(p):
    return 1 - (p**2 + (1-p)**2)

In [ ]:
def best_continious_split(values, targets, criterion, plot=False):
    sorted_pairs = sorted(list(zip(values, targets)), key = lambda x: x[0])

    n = len(sorted_pairs)
    p = sum(targets) / n
    node_criterion = criterion(p)

    best_criterion = node_criterion
    best_split = None

    split_vals = []
    criterion_vals = []

    ones = 0
    total_ones = sum(targets)

    for i in tqdm(range(1, len(sorted_pairs))):
        split_val = (sorted_pairs[i-1][0] + sorted_pairs[i][0]) / 2
        ones += sorted_pairs[i-1][1]

        p_left = ones / i
        p_right = (total_ones - ones) / (n-i)

        split_criterion = i / n * criterion(p_left) + (n-i) / n * criterion(p_right)

        split_vals.append(split_val)
        criterion_vals.append(split_criterion)

        if split_criterion < best_criterion:
            best_criterion = split_criterion
            best_split = split_val

    if plot:
        plt.plot(split_vals, criterion_vals)

    return (best_split, best_criterion)

In [ ]:
def find_best_split(df, feature_col, target_col):
    print('Entropy:')
    display(best_continious_split(df[feature_col], df[target_col], entropy, plot=True))
    plt.show()
    print('Gini Impurity:')
    display(best_continious_split(df[feature_col], df[target_col], gini_impurity, plot=True))
    plt.show()

In [ ]:
find_best_split(train, 'person_income', target_col)

In [ ]:
find_best_split(train[(train['person_income'] < 72652.0) & (train['person_income'] >= 42609.5)], 'person_income', target_col)

In [ ]:
x_cols = train.drop(target_col, axis=1).columns
y_col = target_col

In [ ]:
X_train = train[x_cols]
y_train = train[y_col]

X_test = test[x_cols]
y_test = test[y_col]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score, \
                            confusion_matrix,\
                            ConfusionMatrixDisplay, \
                            accuracy_score, \
                            precision_score, \
                            recall_score, \
                            f1_score

In [ ]:
train_preds = dt.predict(X_train)
test_preds = dt.predict(X_test)

In [ ]:
train_probas = dt.predict_proba(X_train)[:, 1]
test_probas = dt.predict_proba(X_test)[:, 1]

In [ ]:
def show_metrics(true, preds, probas=None):

    print(f"Accuracy: {accuracy_score(true, preds):.2}")
    print(f"Precision: {precision_score(true, preds):.2}")
    print(f"Recall: {recall_score(true, preds):.2}")
    print(f"F1 score: {f1_score(true, preds):.2}")
    print(f"ROC AUC score (predict): {roc_auc_score(true, preds):.2}")

    if probas is not None:
        print(f"ROC AUC score (predict_proba): {roc_auc_score(true, probas):.2}")

    ConfusionMatrixDisplay.from_predictions(true, preds)

In [ ]:
show_metrics(y_train, train_preds, train_probas)

In [ ]:
show_metrics(y_test, test_preds, test_probas)

In [ ]:
print('Количество листьев:', dt.get_n_leaves())
print('Глубина:', dt.get_depth())

In [ ]:
print('Уникальные ответы:')
print(set(dt.predict_proba(X_test)[:, 1]))

In [ ]:
from tqdm.notebook import tqdm

Переберём параметр - количество листьев и посмотрим на изменение метрик качества

In [ ]:
n_leaves = []
train_roc_auc_scores = []
test_roc_auc_scores = []

for n in tqdm(range(2, 200, 2)):
    dt = DecisionTreeClassifier(max_leaf_nodes=n)
    dt.fit(X_train, y_train)

    n_leaves.append(n)
    train_roc_auc_scores.append(roc_auc_score(y_train, dt.predict_proba(X_train)[:, 1]))
    test_roc_auc_scores.append(roc_auc_score(y_test, dt.predict_proba(X_test)[:, 1]))

plt.plot(n_leaves, train_roc_auc_scores, label='Train')
plt.plot(n_leaves, test_roc_auc_scores, label='Test')
plt.xlabel('Максимальное количество листов в дереве')
plt.ylabel('ROC AUC ')
plt.title('Зависимость ROC AUC от сложности дерева на Train и Test выборках')
plt.legend();

In [ ]:
# Выберим то количество листов, которое соответствует максмимальной метрике на test
max_index = test_roc_auc_scores.index(max(test_roc_auc_scores))
best_n_leaves = n_leaves[max_index]

In [ ]:
best_n_leaves

In [ ]:
dt = DecisionTreeClassifier(max_leaf_nodes=best_n_leaves)
dt.fit(X_train, y_train)

In [ ]:
train_preds = dt.predict(X_train)
test_preds = dt.predict(X_test)

In [ ]:
train_probas = dt.predict_proba(X_train)[:, 1]
test_probas = dt.predict_proba(X_test)[:, 1]

In [ ]:
show_metrics(y_train, train_preds, train_probas)

In [ ]:
show_metrics(y_test, test_preds, test_probas)

# Прунинг

In [ ]:
path = dt.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker="o", drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")

In [ ]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)
print(
    "Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
        clfs[-1].tree_.node_count, ccp_alphas[-1]
    )
)

In [ ]:
clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]

node_counts = [clf.tree_.node_count for clf in clfs]
depth = [clf.tree_.max_depth for clf in clfs]
fig, ax = plt.subplots(2, 1)
ax[0].plot(ccp_alphas, node_counts, marker="o", drawstyle="steps-post")
ax[0].set_xlabel("alpha")
ax[0].set_ylabel("number of nodes")
ax[0].set_title("Number of nodes vs alpha")
ax[1].plot(ccp_alphas, depth, marker="o", drawstyle="steps-post")
ax[1].set_xlabel("alpha")
ax[1].set_ylabel("depth of tree")
ax[1].set_title("Depth vs alpha")
fig.tight_layout()

In [ ]:
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker="o", label="train", drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker="o", label="test", drawstyle="steps-post")
ax.legend()
plt.show()

### Домашка

1) По аналогии с функцией best_continious_split, напишите функцию best_categorical_split (которая перебирает все возможные комбинации категорий для сплита), если количество категорий больше 5, то использует стратегию one vs rest

P.S Возможно best_continious_split придётся переписать, убрав предположение о том, что все значения в фиче разные

2) Реализовать собственный класс дерева решения для бинарной классификации

3) Протестировать его на датасете одобрения кредита (approval rate)

4) Сравнить качество работы модели при разных критериях разбиения (gini vs entropy)

5) Сравнить качество с sklearn деревом

In [ ]:
# Используя функции нахождения сплита, реализуйте свой класс MyDecisionTreeClassifier
# Для бинарной классификации, который может использовать в качестве критерия
# gini impurity и entropy на выбор

class MyDecisionTreeClassifier:
    def __init__(self, criterion='gini', max_leaves=None, min_samples_leaf=1):
        '''
        Конструктор класса
        criterion -- критерий может быть либо 'gini', либо 'entropy'
        max_leaves -- максимальное количество листо в дереве, если None -- то дерево не ограничено
        min_samples_leaf -- минимальное количество объектов в листе
        '''
        self.criterion = criterion
        self.max_leaves = max_leaves
        self.min_samples_leaf = min_samples_leaf
        self.root = {
            'feature_name': None,
            'threshold ': None,
            'left_child': None,
            'right_child': None,
            'answer': None
        } # корневой узел

    def fit(X_train, y_train):
        '''
        Функция обучения -- рекурсивно проводит разбиения и выбирает наилучшее
        '''
        pass

    def predict(X):
        '''
        Функция предсказания класса, для каждого наблюдения проходит по дереву, пока не попадёт в лист
        В зависимости от листа, в который попало наблюдение делается предсказание (наиболее часто встречающийся класс)
        '''
        pass

    def predict_proba(X):
        '''
        Функция предсказания вероятности класса, для каждого наблюдения проходит по дереву, пока не попадёт в лист
        В зависимости от листа, в который попало наблюдение делается предсказание для каждого класса:
        Его доля в листе
        '''
        pass

# P.S. При ограничении на количество листьев, возможно нужно подумать над стратегией отбора листьев. Наивный вариант рекурсивно идти в порядке обхода дерева возможно не самый оптимальный